In [1]:
!docker login -u mariembouhadda -p sS58169362

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/jupyter/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [47]:
%%writefile Dockerfile
FROM python:3.7

#RUN git clone https://github.com/meriembouhadda/hello_world.git

COPY hello_world_launcher.py .
COPY hello_world.py . 
COPY requirements.txt .
COPY ./ ./
RUN pip install -r requirements.txt
#ARG project_id='kheopsys-data-lab'
#ARG bucket='ml-pipeline-309409_bucket'
#using bash we run the deploy script we just copied in here
ENTRYPOINT ["python","hello_world_launcher.py"]
#,"--project","ml-pipeline-309409","--bucket","ml-pipeline-309409_bucket"]

Overwriting Dockerfile


In [ ]:
!docker build -t mariembouhadda/hello-world-test .

Sending build context to Docker daemon  506.4kB
Step 1/7 : FROM python:3.7
 ---> d475516974f2
Step 2/7 : COPY hello_world_launcher.py .
 ---> Using cache
 ---> 362593ac25cf
Step 3/7 : COPY hello_world.py .
 ---> Using cache
 ---> 170874d4dbc9
Step 4/7 : COPY requirements.txt .
 ---> Using cache
 ---> badcfe67076e
Step 5/7 : COPY ./ ./
 ---> f2907d7e6da9
Step 6/7 : RUN pip install -r requirements.txt
 ---> Running in 1b68f2e3f609
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 KB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.6/636.6 KB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.

In [ ]:
!docker push mariembouhadda/hello-world-test

In [4]:
import kfp
import kfp.components as comp

In [20]:
@kfp.dsl.component
def hello_world():
    # Defining component configuration
    hello_component = kfp.dsl.ContainerOp(
        name='hello-world',
        image='docker.io/mariembouhadda/hello-world:latest',
        command=['python', 'hello_world.py'],
        )
    return hello_component

In [21]:
@kfp.dsl.pipeline(
  name="hello world script",
  description="hello world script"
)
def hi():
    hello = hello_world()
    hello.execution_options.caching_strategy.max_cache_staleness = "P0D"


In [22]:
kfp.compiler.Compiler().compile(hi, 'hello_world.zip')

In [23]:
client = kfp.Client(host='https://21306363b1ebd301-dot-us-central1.pipelines.googleusercontent.com')

In [24]:
EXPERIMENT_NAME = 'hello world test'
experiment = client.create_experiment(name=EXPERIMENT_NAME)

In [25]:
run = client.run_pipeline(experiment.id, 'hello world test', 'hello_world.zip')

In [12]:
#!pip install db-dtypes

In [13]:
#import db_dtypes